In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

def calculate_features(df):
    """
    1つのデータファイル(DataFrame)から特徴量を計算する関数。
    """
    features = {}
    
    # 1. 3軸合成したベクトルマグニチュードを計算
    df['magnitude'] = np.sqrt(df['x']**2 + df['y']**2 + df['z']**2)
    
    # ★★★ 変更点1: x, y, z の値を足した新しい特徴量 'sum_xyz' を追加 ★★★
    df['sum_xyz'] = df['x'] + df['y'] + df['z']
    
    # 特徴量を計算する対象のカラム
    # ('x', 'y', 'z' の各軸 + 3軸合成の 'magnitude' + 3軸の和 'sum_xyz')
    # ★★★ 変更点2: 計算対象に 'sum_xyz' を追加 ★★★
    columns_to_process = ['x', 'y', 'z', 'magnitude', 'sum_xyz']
    
    for col in columns_to_process:
        signal = df[col]
        
        # 2. 時間領域の統計的特徴量
        features[f'{col}_mean'] = signal.mean()
        features[f'{col}_std'] = signal.std()
        features[f'{col}_var'] = signal.var()
        features[f'{col}_min'] = signal.min()
        features[f'{col}_max'] = signal.max()
        features[f'{col}_median'] = signal.median()
        features[f'{col}_skew'] = skew(signal)
        features[f'{col}_kurtosis'] = kurtosis(signal)
        features[f'{col}_rms'] = np.sqrt(np.mean(signal**2)) # Root Mean Square
        
        # 3. 周波数領域の特徴量 (FFT)
        # サンプリング周期を計算 (データが等間隔と仮定)
        sampling_interval = df['time'].diff().mean()
        if pd.isna(sampling_interval) or sampling_interval == 0:
            sampling_rate = 1 # デフォルト値
        else:
            sampling_rate = 1 / sampling_interval

        # FFTを実行
        n = len(signal)
        fft_vals = np.fft.fft(signal)
        fft_freq = np.fft.fftfreq(n, d=1/sampling_rate)
        
        # 正の周波数成分のみを対象にする
        fft_vals = fft_vals[:n//2]
        fft_freq = fft_freq[:n//2]
        fft_magnitude = np.abs(fft_vals)
        
        # 0Hz成分（直流成分）を除いて、最もパワフルな周波数（＝歩行などの主要な周期）を求める
        if len(fft_magnitude) > 1:
            dominant_freq_idx = np.argmax(fft_magnitude[1:]) + 1
            features[f'{col}_dominant_freq'] = fft_freq[dominant_freq_idx]
            features[f'{col}_dominant_freq_magnitude'] = fft_magnitude[dominant_freq_idx]
        else:
            features[f'{col}_dominant_freq'] = 0
            features[f'{col}_dominant_freq_magnitude'] = 0
            
    return features

# --- メイン処理 ---
root_dir = './'
all_features_list = []

print("特徴量計算を開始します...")

# 'work'ディレクトリ以下を探索
for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        # 前処理済みのCSVファイルのみを対象
        if filename.endswith('_processed.csv'):
            file_path = os.path.join(dirpath, filename)
            
            try:
                # データを読み込み
                df = pd.read_csv(file_path)
                
                # データが空、または短すぎる場合はスキップ
                if df.empty or len(df) < 10:
                    print(f"警告: {file_path} はデータが少ないためスキップします。")
                    continue
                
                # 特徴量を計算
                features = calculate_features(df)
                
                # どのファイルのデータか、また正解ラベルは何かを記録
                features['filename'] = filename
                # 正解ラベルをディレクトリ名から取得 (例: work/aruki/aruki1 -> aruki)
                label = os.path.basename(os.path.dirname(dirpath))
                features['label'] = label
                
                all_features_list.append(features)
                print(f"完了: {file_path}")

            except Exception as e:
                print(f"エラー: {file_path} の処理中にエラーが発生しました - {e}")

# リストをDataFrameに変換
features_df = pd.DataFrame(all_features_list)

# カラムの順序を整理 (ファイル名とラベルを先頭に)
cols = features_df.columns.tolist()
cols = cols[-2:] + cols[:-2]
features_df = features_df[cols]

# 結果をCSVファイルとして保存
output_path = 'sensor_features.csv'
features_df.to_csv(output_path, index=False)

print(f"\nすべての特徴量計算が完了しました。結果は '{output_path}' に保存されました。")